In [19]:
import pandas as pd
import numpy as np
import plotly.express as px
import scipy.stats as stats
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


In [4]:
df = pd.read_csv("data/ratings_clean_names.csv")
df.head(2)

userId  rating  movieId   timestamp       title_safe
0    2038     2.0       18  1305046590  Four Rooms(5.0)
1    2858     4.0       18   856639289  Four Rooms(5.0)

In [5]:
# pivot to have users as rows and movies as columns
pivoted = df.pivot(index='userId', columns='title_safe', values='rating')

pivoted=pivoted.notnull().astype("int")

In [8]:
supports = apriori(pivoted, min_support=0.1,max_len=3, use_colnames=True)

/Users/remycazabet/opt/anaconda3/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [9]:
supports.sort_values(by="support", ascending=False).head(10)

support                           itemsets
45  0.448844              (Pulp Fiction(680.0))
74  0.444600  (The Shawshank Redemption(278.0))
26  0.438121               (Forrest Gump(13.0))
75  0.400648  (The Silence of the Lambs(274.0))
36  0.348989             (Jurassic Park(329.0))
5   0.336926                 (Apollo 13(568.0))
63  0.328326             (The Fugitive(5503.0))
18  0.325087        (Dances with Wolves(581.0))
9   0.324472                    (Batman(268.0))
12  0.319725                (Braveheart(197.0))

In [12]:
supports["len"] = supports["itemsets"].apply(lambda x: len(x))

In [16]:
pd.set_option("display.max_colwidth", None)
supports[supports["len"]==2].sort_values(by="support", ascending=False).head(10)

support  \
616  0.279739   
479  0.274210   
617  0.269910   
484  0.261532   
498  0.253323   
730  0.250754   
247  0.248688   
499  0.248632   
172  0.245002   
166  0.244611   

                                                               itemsets  len  
616              (Pulp Fiction(680.0), The Shawshank Redemption(278.0))    2  
479                          (Jurassic Park(329.0), Forrest Gump(13.0))    2  
617              (Pulp Fiction(680.0), The Silence of the Lambs(274.0))    2  
484                           (Pulp Fiction(680.0), Forrest Gump(13.0))    2  
498               (Forrest Gump(13.0), The Shawshank Redemption(278.0))    2  
730  (The Shawshank Redemption(278.0), The Silence of the Lambs(274.0))    2  
247                          (Batman(268.0), Dances with Wolves(581.0))    2  
499               (Forrest Gump(13.0), The Silence of the Lambs(274.0))    2  
172                       (Apollo 13(568.0), Dances with Wolves(581.0))    2  
166                                   (Batman(268.0), Apollo 13(568.0))    2

# Analysis 
We observe that the movies that appear the most frequently together are also the most frequent movies, moslty.  This is clearly a bias that we would like to correct. This correction comes from association rules

In [20]:
# compute association rules
rules = association_rules(supports, min_threshold=0.5)
rules.head(2)


antecedents                           consequents  \
0        (Addams Family Values(2758.0))  (Ace Ventura: Pet Detective(3049.0))   
1  (Ace Ventura: Pet Detective(3049.0))                      (Aladdin(812.0))   

   antecedent support  consequent support   support  confidence      lift  \
0            0.145314            0.254942  0.108288    0.745196  2.922997   
1            0.254942            0.269965  0.175360    0.687842  2.547891   

   leverage  conviction  zhangs_metric  
0  0.071241    2.924042       0.769740  
1  0.106535    2.338672       0.815398

In [21]:
rules.sort_values(by="lift", ascending=False).head(10)

antecedents  \
8815  (The Lord of the Rings: The Return of the King(122.0), The Lord of the Rings: The Fellowship of the Ring(120.0))   
8820                                                                    (The Lord of the Rings: The Two Towers(121.0))   
8816              (The Lord of the Rings: The Return of the King(122.0), The Lord of the Rings: The Two Towers(121.0))   
8819                                                        (The Lord of the Rings: The Fellowship of the Ring(120.0))   
8818                                                            (The Lord of the Rings: The Return of the King(122.0))   
8817          (The Lord of the Rings: The Two Towers(121.0), The Lord of the Rings: The Fellowship of the Ring(120.0))   
7866                                                                         (Forrest Gump(13.0), Pretty Woman(114.0))   
7870                                                                                                    (Ghost(251.0))   
842                                                                     (The Lord of the Rings: The Two Towers(121.0))   
843                                                         (The Lord of the Rings: The Fellowship of the Ring(120.0))   

                                                                                                           consequents  \
8815                                                                    (The Lord of the Rings: The Two Towers(121.0))   
8820  (The Lord of the Rings: The Return of the King(122.0), The Lord of the Rings: The Fellowship of the Ring(120.0))   
8816                                                        (The Lord of the Rings: The Fellowship of the Ring(120.0))   
8819              (The Lord of the Rings: The Return of the King(122.0), The Lord of the Rings: The Two Towers(121.0))   
8818          (The Lord of the Rings: The Two Towers(121.0), The Lord of the Rings: The Fellowship of the Ring(120.0))   
8817                                                            (The Lord of the Rings: The Return of the King(122.0))   
7866                                                                                                    (Ghost(251.0))   
7870                                                                         (Forrest Gump(13.0), Pretty Woman(114.0))   
842                                                         (The Lord of the Rings: The Fellowship of the Ring(120.0))   
843                                                                     (The Lord of the Rings: The Two Towers(121.0))   

      antecedent support  consequent support   support  confidence      lift  \
8815            0.135262            0.158997  0.113035    0.835673  5.255905   
8820            0.158997            0.135262  0.113035    0.710924  5.255905   
8816            0.125768            0.171116  0.113035    0.898757  5.252329   
8819            0.171116            0.125768  0.113035    0.660574  5.252329   
8818            0.172624            0.127890  0.113035    0.654804  5.120055   
8817            0.127890            0.172624  0.113035    0.883843  5.120055   
7866            0.141629            0.150229  0.101977    0.720032  4.792894   
7870            0.150229            0.141629  0.101977    0.678810  4.792894   
842             0.158997            0.171116  0.127890    0.804355  4.700649   
843             0.171116            0.158997  0.127890    0.747389  4.700649   

      leverage  conviction  zhangs_metric  
8815  0.091528    5.117863       0.936397  
8820  0.091528    2.991384       0.962824  
8816  0.091514    8.187049       0.926079  
8819  0.091514    2.575622       0.976745  
8818  0.090958    2.526421       0.972580  
8817  0.090958    7.122901       0.922693  
7866  0.080700    3.035238       0.921929  
7870  0.080700    2.672476       0.931260  
842   0.100683    4.236684       0.936101  
843   0.100683    3.329242       0.949787

In [23]:
rules.sort_values(by="zhangs_metric", ascending=False).head(10)

antecedents  \
4040                                             (Batman(268.0))   
2030                                             (Batman(268.0))   
3942                                             (Batman(268.0))   
3948                                             (Batman(268.0))   
2722                                             (Batman(268.0))   
3985                                             (Batman(268.0))   
4014                                             (Batman(268.0))   
1080                                             (Batman(268.0))   
8819  (The Lord of the Rings: The Fellowship of the Ring(120.0))   
8818      (The Lord of the Rings: The Return of the King(122.0))   

                                                                                                   consequents  \
4040                                                               (Batman Forever(414.0), True Lies(36955.0))   
2030                                                                   (Batman Forever(414.0), Aladdin(812.0))   
3942                                                        (Batman Forever(414.0), Dances with Wolves(581.0))   
3948                                               (Batman Forever(414.0), Die Hard: With a Vengeance(1572.0))   
2722                                                                 (Batman Forever(414.0), Apollo 13(568.0))   
3985                                                              (Pulp Fiction(680.0), Batman Forever(414.0))   
4014                                                             (Batman Forever(414.0), The Fugitive(5503.0))   
1080                                               (Batman Forever(414.0), Ace Ventura: Pet Detective(3049.0))   
8819      (The Lord of the Rings: The Return of the King(122.0), The Lord of the Rings: The Two Towers(121.0))   
8818  (The Lord of the Rings: The Two Towers(121.0), The Lord of the Rings: The Fellowship of the Ring(120.0))   

      antecedent support  consequent support   support  confidence      lift  \
4040            0.324472            0.190104  0.183570    0.565749  2.975998   
2030            0.324472            0.171563  0.165084    0.508778  2.965553   
3942            0.324472            0.189154  0.181838    0.560413  2.962727   
3948            0.324472            0.169943  0.163186    0.502926  2.959380   
2722            0.324472            0.190718  0.182900    0.563683  2.955582   
3985            0.324472            0.181336  0.173461    0.534596  2.948096   
4014            0.324472            0.177817  0.169887    0.523580  2.944480   
1080            0.324472            0.179996  0.171786    0.529432  2.941362   
8819            0.171116            0.125768  0.113035    0.660574  5.252329   
8818            0.172624            0.127890  0.113035    0.654804  5.120055   

      leverage  conviction  zhangs_metric  
4040  0.121886    1.865040       0.982903  
2030  0.109417    1.686483       0.981151  
3942  0.120463    1.844562       0.980675  
3948  0.108044    1.669886       0.980110  
2722  0.121017    1.854804       0.979467  
3985  0.114623    1.759038       0.978195  
4014  0.112190    1.725752       0.977579  
1080  0.113382    1.742584       0.977046  
8819  0.091514    2.575622       0.976745  
8818  0.090958    2.526421       0.972580

In [24]:
# find the best antecedents for a give consequents
rules[rules["consequents"]=={"Aladdin(812.0)"}].sort_values(by="lift", ascending=False).head(10)

antecedents  \
2032               (Beauty and the Beast(10020.0), Batman Forever(414.0))   
891   (Beauty and the Beast(10020.0), Ace Ventura: Pet Detective(3049.0))   
2227    (Beauty and the Beast(10020.0), Clear and Present Danger(9331.0))   
2286               (The Lion King(8587.0), Beauty and the Beast(10020.0))   
2251             (Beauty and the Beast(10020.0), Dumb and Dumber(8467.0))   
2236                (Crimson Tide(8963.0), Beauty and the Beast(10020.0))   
2270       (Beauty and the Beast(10020.0), Star Trek: Generations(193.0))   
2275                    (Beauty and the Beast(10020.0), Stargate(2164.0))   
2246  (Beauty and the Beast(10020.0), Die Hard: With a Vengeance(1572.0))   
2306                  (Beauty and the Beast(10020.0), Waterworld(9804.0))   

           consequents  antecedent support  consequent support   support  \
2032  (Aladdin(812.0))            0.154082            0.269965  0.136714   
891   (Aladdin(812.0))            0.153412            0.269965  0.135765   
2227  (Aladdin(812.0))            0.139674            0.269965  0.123031   
2286  (Aladdin(812.0))            0.147102            0.269965  0.129566   
2251  (Aladdin(812.0))            0.123925            0.269965  0.109125   
2236  (Aladdin(812.0))            0.117949            0.269965  0.103652   
2270  (Aladdin(812.0))            0.133810            0.269965  0.117447   
2275  (Aladdin(812.0))            0.133698            0.269965  0.117000   
2246  (Aladdin(812.0))            0.137775            0.269965  0.120351   
2306  (Aladdin(812.0))            0.116106            0.269965  0.101419   

      confidence      lift  leverage  conviction  zhangs_metric  
2032    0.887278  3.286636  0.095117    6.476416       0.822465  
891     0.884965  3.278070  0.094349    6.346219       0.820875  
2227    0.880848  3.262817  0.085324    6.126901       0.806109  
2286    0.880790  3.262602  0.089853    6.123921       0.813105  
2251    0.880577  3.261814  0.075670    6.113007       0.791510  
2236    0.878788  3.255187  0.071810    6.022786       0.785440  
2270    0.877713  3.251205  0.081323    5.969839       0.799388  
2275    0.875104  3.241543  0.080906    5.845160       0.798226  
2246    0.873531  3.235713  0.083156    5.772421       0.801356  
2306    0.873497  3.235589  0.070074    5.770882       0.781697

# Analysis
We observe that by using association rules, we obtain more interesting results: 
* Watching a lord of the ring movies is associated with watching the others
* Similar for Batman
* Watching Aladdin is well predicted by watching Beauty and the Beast, the disney movie released the previous year
* Note that while the other movies seem somewhat unrelated, they all belong to movies dating from the same period... So probably like by people of a certain age...
